## Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results, and you'll learn methods to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Analyze the results of regression and R-squared and adjusted-R-squared 
* Understand and apply forward and backward predictor selection

## The Boston Housing Data once more

We pre-processed the Boston Housing Data the same way we did before:

- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

## Perform stepwise selection

The code for stepwise selection is copied below.

In [3]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [4]:
boston_target = pd.DataFrame(boston.target)
result = stepwise_selection(boston_features, boston_target, verbose = True)
print(result)


Add  LSTAT                          with p-value 9.27989e-122
Add  RM                             with p-value 1.98621e-16
Add  PTRATIO                        with p-value 2.5977e-12
Add  DIS                            with p-value 2.85496e-09
Add  B                              with p-value 2.77572e-06
Add  TAX_(0, 270]                   with p-value 0.000855799
Add  CHAS                           with p-value 0.00151282
Add  INDUS                          with p-value 0.00588575
['LSTAT', 'RM', 'PTRATIO', 'DIS', 'B', 'TAX_(0, 270]', 'CHAS', 'INDUS']


In [5]:
boston_features.columns

Index(['CRIM', 'INDUS', 'CHAS', 'RM', 'AGE', 'DIS', 'PTRATIO', 'B', 'LSTAT',
       'RAD_(0, 6]', 'RAD_(6, 24]', 'TAX_(0, 270]', 'TAX_(270, 360]',
       'TAX_(360, 712]'],
      dtype='object')

### Build the final model again in Statsmodels

In [6]:
boston_features_fin = boston_features.drop(['CRIM', 'AGE', 'RAD_(0, 6]', 'RAD_(6, 24]','TAX_(270, 360]', 'TAX_(360, 712]'], axis=1)
boston_features_fin.head()

,INDUS,CHAS,RM,DIS,PTRATIO,B,LSTAT,"TAX_(0, 270]"
0,-1.704344,0.0,6.575,0.542096,-1.443977,1.000000,-1.275260,0
1,-0.263239,0.0,6.421,0.623954,-0.230278,1.000000,-0.263711,1
2,-0.263239,0.0,7.185,0.623954,-0.230278,0.989737,-1.627858,1
3,-1.778965,0.0,6.998,0.707895,0.165279,0.994276,-2.153192,1
4,-1.778965,0.0,7.147,0.707895,0.165279,1.000000,-1.162114,1


In [12]:
model = sm.OLS(boston_target, boston_features_fin).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      0   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     1879.
Date:                Sat, 10 Nov 2018   Prob (F-statistic):               0.00
Time:                        21:28:42   Log-Likelihood:                -1462.8
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     498   BIC:                             2975.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
INDUS           -0.7985      0.335     -2.387      0.017      -1.456      -0.141
CHAS             2.7534      0.792      3.476      0.001       1.197       4.309
RM               3.4584      0.136     25.414      0.000       3.191       3.726
DIS             -8.4164      1.258     -6.690      0.000     -10.888      -5.945
PTRATIO         -1.3066      0.226     -5.785      0.000      -1.750      -0.863
B                4.5405      0.858      5.290      0.000       2.854       6.227
LSTAT           -5.2619      0.257    -20.514      0.000      -5.766      -4.758
TAX_(0, 270]     1.4567      0.553      2.636      0.009       0.371       2.543
==============================================================================
Omnibus:                      126.200   Durbin-Watson:                   1.071
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              614.720
Skew:                           1.005   Prob(JB):                    3.28e-134
Kurtosis:                       8.012   Cond. No.                         43.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Where our stepwise procedure mentions that "CHAS" was added with a p-value of 0.00151282, but our statsmodels output returns a p-value of 0.000. What is the intuition behind this?

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [7]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression


linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 5)
selector = selector.fit(boston_features_fin, boston_target)

C:\Users\Stee\Anaconda3good\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
selector.support_

array([False,  True,  True,  True, False,  True,  True, False])

Fit the linear regression model again using the 5 columns selected

In [43]:
np.array(boston_features_fin.columns)

array(['INDUS', 'CHAS', 'RM', 'DIS', 'PTRATIO', 'B', 'LSTAT',
       'TAX_(0, 270]'], dtype=object)

In [50]:
features_ranked_names = []
#boston_features_fin.columns
#len(boston_features_fin.columns), len(selector.support_)
for i in range(len(selector.support_)):
    if selector.support_[i]:
        features_ranked_names.append(boston_features_fin.columns[i])

features_ranked_names

['CHAS', 'RM', 'DIS', 'B', 'LSTAT']

In [54]:
boston_features_ranked = boston_features_fin[features_ranked_names] #'CHAS', 'RM', 'DIS', 'B', 'LSTAT'

In [57]:
test = boston_features_fin.columns[selector.support_]
test

Index(['CHAS', 'RM', 'DIS', 'B', 'LSTAT'], dtype='object')

In [55]:
model = sm.OLS(boston_target, boston_features_ranked).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      0   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     2634.
Date:                Sun, 11 Nov 2018   Prob (F-statistic):               0.00
Time:                        10:48:34   Log-Likelihood:                -1496.5
No. Observations:                 506   AIC:                             3003.
Df Residuals:                     501   BIC:                             3024.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CHAS           2.9363      0.834      3.523      0.000       1.299       4.574
RM             3.3710      0.135     25.056      0.000       3.107       3.635
DIS           -6.6241      1.086     -6.097      0.000      -8.759      -4.489
B              4.5876      0.912      5.032      0.000       2.796       6.379
LSTAT         -6.2934      0.229    -27.528      0.000      -6.743      -5.844
==============================================================================
Omnibus:                       82.438   Durbin-Watson:                   1.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              264.051
Skew:                           0.744   Prob(JB):                     4.59e-58
Kurtosis:                       6.211   Cond. No.                         36.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
linreg.fit(boston_features_ranked, boston_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Now, predict $\hat y$ using your model. you can use `.predict()` in scikit-learn

In [60]:
y_hat = linreg.predict(boston_features_ranked)

Now, using the formulas of R-squared and adjusted-R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted-R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

## Level up - Optional

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted-R-squared!
- Tweak the code in the `stepwise_selection()`-function written above to just perform forward selection based on the p-value.

## Summary
Great! You now performed your own feature selection methods!